In [12]:
import pandas as pd
import numpy as np
import math
from time import sleep as fallasleep
import os, sys

In [8]:
import krakenex
from pykrakenapi import KrakenAPI

In [14]:
color_box = ['#1f77b4','#ff7f0e','#2ca02c','#d62728',
             '#9467bd','#8c564b','#e377c2','#7f7f7f',
             '#bcdb22','#17becf']

In [15]:
api = krakenex.API()
k = KrakenAPI(api)
coins = ["ETH", "BTC", "XRP", "ADA", "DOT"]
gapmin= 1440
coins = [coin+"USD" for coin in coins]
df = {}
colormapper = {}
for _,coin in enumerate(coins):
    data, last = k.get_ohlc_data(coin, interval = gapmin, ascending = True)
    df[coin]=data
    colormapper[coin] = color_box[_]
    fallasleep(1)


##### Recall from `causation.ipynb` that price of the Bitcoin can be predicted with using all other 4 coins.